In [4]:
import base64

import requests

from lxml import html as lhtml

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool

In [5]:
from functools import reduce
def concatenate(a,b):
    return a + b


In [6]:
authors_id = [15753,19769,15844,19768,20948,43102,19073,19071,20416,17675,19076,20700,28169,19805,20010,16471,17717,19806,43162,20935,16340,20282,19075,15764,16297,20591,26822,19072,19725,20542,17896,19070,20487,19727,20540]

In [21]:

def get_urls(author_id):
    urls = []
    MAX_PAGES_ON_EYE = 15
    for i in tqdm(range(1, MAX_PAGES_ON_EYE)):
        page = requests.get('https://www.respublica.ru/authors/{}'.format(author_id),data = {'a_p': True,
                                                                              'page': i,
                                                                              'ajax_append_page': True})
        if not page.ok:
            print(page.status_code, i)
        tree = lhtml.fromstring(page.content)
        page_urls = tree.xpath('//div/a[@alt][@href][@title]/attribute::href')
        if not page_urls:
            break
        urls.extend(page_urls)
    #len(urls)
    return urls
#all_urls = list(map(get_urls,tqdm(authors_id)))
with ThreadPool(10) as pool:
    all_urls = pool.map(get_urls,tqdm(authors_id))
#pool.join()
all_urls = reduce(concatenate,all_urls)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\User\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 78, in run
    instances = self.get_instances()
  File "C:\Users\User\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 58, in get_instances
    return [i for i in self.tqdm_cls._instances.copy()
  File "C:\Users\User\Anaconda3\lib\_weakrefset.py", line 92, in copy
    return self.__class__(self)
  File "C:\Users\User\Anaconda3\lib\_weakrefset.py", line 50, in __init__
    self.update(data)
  File "C:\Users\User\Anaconda3\lib\_weakrefset.py", line 119, in update
    for element in other:
  File "C:\Users\User\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [22]:

with ThreadPool(10) as pool:
    urls = pool.map(lambda x:'https://www.respublica.ru' + x, all_urls)


In [23]:
len(urls)#urls = list(map(lambda x:'https://www.respublica.ru' + x, urls))

2461

In [11]:
def process_page(url):
    page = requests.get(url)
    tree = lhtml.fromstring(page.content)
    
    book_id = tree.xpath('//div/span[@itemprop="sku"]/text()')
    book_id = ''.join(book_id)
    title = tree.xpath('//h1[@itemprop="name"]/text()')
    title = ''.join(title)
    author = tree.xpath('//div/a[@itemprop="brand"]/text()')
    author = ''.join(author)
    preview = tree.xpath('//div/a[@class="download-pdf"]/attribute::href')
    preview = ('https://www.respublica.ru'+''.join(preview))
    image = tree.xpath('//link[@href][@rel="image_src"]/attribute::href')
    image = ''.join(image)
    description = tree.xpath('//div[@itemprop="description"]/p/text()')
    description = ''.join(description)
    price = tree.xpath('//div[@class="rd-page-product__price"]/span/text()')
    price = ''.join(price)
    price_old = tree.xpath('//div[@class="rd-page-product__price-old"]/span/text()')
    price_old = ''.join(price_old)
    category = tree.xpath('//div[contains(@class, "rd-page-product__breadcrumbs")]//span[@itemprop="name"]/text()')
    category = ';'.join(category)
    instock = tree.xpath('//div[@class="rd-page-product__buttons"]/a[contains(@class, rd-page-product__buy)]/attribute::class')
    instock = list(map(str,instock[0].split()))
    if instock[1]=='rd-page-product__buy_status_available':
        instock = True
    else:
        instock = False
    review_count = tree.xpath('//div[@class="rd-rating-stars"]/span/meta[@itemprop="reviewCount"]/attribute::content')
    review_count = ''.join(review_count)
    rating_count = tree.xpath('//div[@class="rd-rating-stars"]/span/meta[@itemprop="ratingCount"]/attribute::content')
    rating_count = ''.join(rating_count)
    rating = tree.xpath('//div[@class="rd-rating-stars"]/span/meta[@itemprop="ratingValue"]/attribute::content')
    rating = ''.join(rating)
    info = {
        "ID": book_id,
        "URL": url,
        "Название": title,
        "Автор": author,
        "Превью": preview,
        "Изображение": image,
        "Описание": description,
        "Цена": price,
        "Цена (старая)": price_old,
        "В наличии": instock,
        "Категория": category,
        "Число отзывов": review_count,
        "Число оценок": rating_count,
        "Оценка": rating,
    }
    index = tree.xpath('//div[@class="rd-page-product__desc-params"]/p/span[contains(@itemprop, "name")]/text()')
    params = tree.xpath('//div[@class="rd-page-product__desc-params"]/p/span[contains(@itemprop, "value")]/text()|//div[@class="rd-page-product__desc-params"]/p/a[contains(@itemprop, "value")]/text()')
    par = {index[i]:params[i] for i in range(len(index))}
    info = {**info,**par}
    return info

In [24]:
import pandas as pd
with ThreadPool(10) as pool:
    result = pool.map(process_page, tqdm(urls))
df = pd.DataFrame(result, index='ID')
df.sort_values(by=['ID'], inplace=True)

with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

TypeError: Index(...) must be called with a collection of some kind, 'ID' was passed

In [26]:
#print(result[::10])

In [25]:
df

,ID,URL,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,...,Иллюстратор,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок
2069,000033,https://www.respublica.ru/knigi/hudozhestvenna...,Много шума из ничего,Уильям Шекспир,https://www.respublica.ru,https://www.respublica.ru/uploads/00/00/00/cq/...,Книга «Много шума из ничего» (Уильям Шекспир) ...,100,140,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2068,000056,https://www.respublica.ru/knigi/hudozhestvenna...,Двенадцатая ночь,Уильям Шекспир,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/24/...,Книга «Двенадцатая ночь» (Уильям Шекспир) от и...,100,140,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1854,021678,https://www.respublica.ru/knigi/hudozhestvenna...,N or M?,Agatha Christie,https://www.respublica.ru,https://www.respublica.ru/uploads/00/00/00/68/...,Tommy & Tuppence are hired to track down warti...,610,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,021679,https://www.respublica.ru/knigi/hudozhestvenna...,The Secret Adversary,Agatha Christie,https://www.respublica.ru,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's first Tommy and Tuppence mys...,610,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1877,021680,https://www.respublica.ru/knigi/hudozhestvenna...,By the Pricking of My Thumbs,Agatha Christie,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/68/...,An old woman in a nursing home speaks of a chi...,510,610,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,532521,https://www.respublica.ru/knigi/hudozhestvenna...,Трагедии,Уильям Шекспир,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/dp/...,"Из века в век люди ищут ответа на вопрос, в че...",740,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2416,532523,https://www.respublica.ru/knigi/detskie-knigi/...,Пеппи Длинныйчулок в стране Веселии,Астрид Линдгрен,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/dp/...,Озорная девчонка Пеппи Длинныйчулок объездила ...,660,,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2415,532529,https://www.respublica.ru/knigi/detskie-knigi/...,Приключения Эмиля из Лённеберги,Астрид Линдгрен,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/dp/...,"Весёлая повесть про Эмиля из Лённеберги, котор...",680,,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,8121280,https://www.respublica.ru/knigi/hudozhestvenna...,Завтрашний день кошки,Бернар Вербер,https://www.respublica.ru,https://www.respublica.ru/uploads/01/00/00/6h/...,Книга «Завтрашний день кошки» (Бернар Вербер) ...,430,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
